## Italian demography dashboard
Data from the [United Nations Data Portal](https://population.un.org/dataportal/home).

In [2]:
import pandas as pd
import plotly.graph_objs as go

import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

import warnings
warnings.filterwarnings("ignore")

In [3]:
# may be needed if using VS
# %pip install jupyter_bokeh

### Request data

In [4]:
# GET DATA OF ITALIAN POPULATION BY 5-YEARS GROUPS, BY GENDER
# FROM UN DATA PORTAL 

def get_data(indicator = 46, # Population by 5-year age groups and sex
            location = 380, # Italy
            start_year = 1990, end_year = 2021):

    # Imports to fetch data
    import requests
    import json

    # Declares the base url for calling the API
    base_url = "https://population.un.org/dataportalapi/api/v1"

    # Creates the target URL, indicators, in this instance
    target = base_url + "/data/indicators/"+str(indicator)+"/locations/"+str(location)+"/start/"+str(start_year)+"/end/"+str(end_year)

    # Get the response, which includes the first page of data as well as information on pagination and number of records
    response = requests.get(target)

    # Converts call into JSON
    j = response.json()

    # Converts JSON into a pandas DataFrame.
    df = pd.json_normalize(j['data']) # pd.json_normalize flattens the JSON to accomodate nested lists within the JSON structure

    # Loop until there are new pages with data
    while j['nextPage'] != None:
        # Reset the target to the next page
        target = j['nextPage']

        #call the API for the next page
        response = requests.get(target)

        # Convert response to JSON format
        j = response.json()

        # Store the next page in a data frame
        df_temp = pd.json_normalize(j['data'])

        # Append next page to the data frame
        df = df.append(df_temp)

    df.timeLabel = df.timeLabel.astype('int')

    if indicator == 19: # if Total Fertility Rate data
        # get median value for each year
        df = df[df.variantLabel=='Median'].reset_index(drop=True)

    return df[['location', 'timeLabel', 'sex', 'ageLabel', 'value']]


### Population pyramid 

In [5]:
df = get_data()
df

,location,timeLabel,sex,ageLabel,value
0,Italy,1990,Male,0-4,1432747.5
1,Italy,1990,Female,0-4,1358892.5
2,Italy,1990,Both sexes,0-4,2791640.0
3,Italy,1990,Male,5-9,1539409.5
4,Italy,1990,Female,5-9,1470743.0
...,...,...,...,...,...
11,Italy,2021,Female,95-99,128010.0
12,Italy,2021,Both sexes,95-99,165877.5
13,Italy,2021,Male,100+,2873.5
14,Italy,2021,Female,100+,15029.5


In [6]:
def human_format(num):
    ''' 
    Function to format numbers as str w/ suffixes in chart ticks
    '''
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude]) # one decimal

In [7]:
def pyramid(year=2005):
    ''' 
    Function to draw the population pyramid for the selected year
    '''
    df = get_data()
    # erase rows with counts for M+F
    df = df[df.sex!='Both sexes'].reset_index(drop=True)

    # use selected year
    df = df[df.timeLabel == year][['sex', 'ageLabel', 'value']]

    # dataframes per gender
    male_df = df[df.sex=='Male'][['ageLabel', 'value']].rename(columns={'value':'M'})
    female_df = df[df.sex=='Female'][['ageLabel', 'value']].rename(columns={'value':'F'})

    # dataframe as "age_range"-"Male_value"-"Female_value"
    df = male_df.merge(female_df, left_on='ageLabel', right_on='ageLabel')

    # create variables for pyramid
    y = df.ageLabel
    x_male = df.M 
    x_female = - df.F # with minus bc left-side of chart

    # initialize figure
    fig = go.Figure()

    # male barchart
    fig.add_trace(go.Bar(
        y=y,
        x=x_male,
        name='Male',
        orientation='h',
        marker_color='#6e44ff'
    ))

    # female barchart
    fig.add_trace(go.Bar(
        y=y,
        x=x_female,
        name='Female',
        orientation='h',
        marker_color='#ef7a85'
    ))

    # chart layout
    fig.update_layout(
        template='plotly_white',
        title='Population Pyramid in Italy for '+str(year),
        title_font_size=24,
        barmode='relative',
        bargap=0.0,
        bargroupgap=0,
        # width=800, 
        height=400,

        # fix axis -> female ticks are neg.values
        xaxis=dict(
            # where to put the ticks (defined after)
            tickvals=[-df.F.max(), -df.F.max()//2, 0, df.M.max()//2, df.M.max()],
            # ticks text using previously defined function
            ticktext=[human_format(df.F.max()), human_format(df.F.max()//2), 0, human_format(df.M.max()//2), human_format(df.M.max())],
            # axis label
            title = 'Population',
            title_font_size=14
        )
    )

    return fig

pyramid() # TODO: to bind with the slider using pn.bind in the template


### Panel commands:

In [8]:
# make DataFrame Pipeline Interactive
idf = df.interactive()
idf.head()

BokehModel(combine_events=True, render_bundle={'docs_json': {'f39818a7-1e49-47b6-a106-0c6f8f87b812': {'defs': …

### Slider

In [9]:
# define panel slider to select the year
year_slider = pn.widgets.IntSlider(name='Year slider', start=int(df.timeLabel.min()), end=int(df.timeLabel.max()), step=1, value=int(df.timeLabel.max()))
year_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'021a527e-bb2c-4a6e-98cc-f43192bf7ebc': {'defs': …

### Population Curve

In [10]:
# countries = ['Italy']

demo_pipeline = (
    idf[
        (idf.timeLabel <= year_slider) &
        (idf.sex == 'Both sexes') # &
        # (idf.location.isin(countries))
    ].groupby(['timeLabel', 'location'])['value'].sum().to_frame().reset_index().sort_values(by='timeLabel').reset_index(drop=True)
)

In [11]:
pop_curve = demo_pipeline.hvplot(x = 'timeLabel', by='location', line_width=2, title="Population curve", color='#6e44ff')
pop_curve 

BokehModel(combine_events=True, render_bundle={'docs_json': {'73079c33-5e74-465f-a56a-19cc1648a243': {'defs': …

### Data table

In [12]:
demo_pipeline = demo_pipeline.rename(columns={'timeLabel':'year', 'value':'population'})

In [13]:
data_table = demo_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
data_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'51f353ef-8864-4462-881e-9be8743e7b5a': {'defs': …

### Total fertility rate

In [14]:
tfr = get_data(indicator = 19, # Total fertility rate
        location = 380 # Italy
        )[['location', 'timeLabel', 'value']]

tfr = tfr.rename(columns={'timeLabel':'year'})

tfr_idf = tfr.interactive()
tfr_pipeline = (
    tfr_idf[
        (tfr_idf.year <= year_slider) 
    ].groupby(['year', 'location'])['value'].mean().to_frame().reset_index().sort_values(by='year').reset_index(drop=True)
)

In [15]:
tfr_plot = tfr_pipeline.hvplot(x = 'year', y='value', by='location', line_width=2, title="Total Fertility Rate (TFR) - Italy", legend=False, color='#ef7a85',  ylim=(tfr.value.min(), tfr.value.max())) 
tfr_plot 

BokehModel(combine_events=True, render_bundle={'docs_json': {'145c322c-2473-4bca-998a-479e9a8c65cf': {'defs': …

### Life expectacy at birth 

In [16]:
life_exp = get_data(indicator = 61, # Life expectancy at birth
        location = 380 # Italy
        )
life_exp = life_exp[life_exp.sex=='Both sexes'][['location', 'timeLabel', 'value']].reset_index(drop=True)

### Scatterplot: life expectation vs TFR per diversi paesi europei

In [17]:
# selected countries IDs
eu_ids = [208, 246, 250, 276, 300, 348, 372, 528, 578, 616, 620, 724, 752, 756, 826]

# add other countries
for id in eu_ids:
    life_exp = life_exp.append(get_data(indicator = 61, # Life expectancy at birth
        location = id))

In [18]:
# add TFR of other countries
tfr = get_data(indicator = 19, # Total fertility rate
        location = 380)
for id in eu_ids:
    tfr = tfr.append(get_data(indicator = 19, # Life expectancy at birth
        location = id))

In [19]:
life_exp = life_exp[['location', 'timeLabel', 'value']].rename(columns={'value':'LifeExpectation'})
tfr = tfr[['location', 'timeLabel', 'value']].rename(columns={'value':'TFR'})

In [20]:
eu_df = tfr.merge(life_exp, left_on=['timeLabel', 'location'], right_on=['timeLabel', 'location'])

In [21]:
eu_df = eu_df.rename(columns={'timeLabel':'year', 'location':'Country'})
eu_df

,Country,year,TFR,LifeExpectation
0,Italy,1990,1.332150,76.9909
1,Italy,1991,1.305087,76.9891
2,Italy,1992,1.297809,77.3642
3,Italy,1993,1.251105,77.6524
4,Italy,1994,1.215847,77.8724
...,...,...,...,...
1651,United Kingdom,2021,1.561956,80.7420
1652,United Kingdom,2021,1.561956,80.7420
1653,United Kingdom,2021,1.561956,80.7422
1654,United Kingdom,2021,1.561956,80.7420


### Total population per year for each selected country

In [22]:
population = get_data()
for id in eu_ids:
    population = population.append(get_data(location = id))
population = population[population.sex=='Both sexes'].groupby(['timeLabel', 'location']).aggregate({'value':'sum'}).reset_index().rename(columns={'timeLabel':'year', 'location':'Country'})

eu_df_pop = eu_df.merge(population, left_on=['year', 'Country'], right_on=['year', 'Country'])
eu_df_pop = eu_df_pop.rename(columns={'value':'population'})

In [23]:
euidf = eu_df_pop.interactive()

tab_pipeline = (
    euidf[
        (euidf.year <= year_slider) 
    ]#.groupby(['timeLabel', 'location'])['value'].sum().to_frame().reset_index().sort_values(by='timeLabel').reset_index(drop=True)
)

In [24]:
data_table2 = tab_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
data_table2

BokehModel(combine_events=True, render_bundle={'docs_json': {'2b07168b-11fd-4178-8dab-5f86bf9b84a2': {'defs': …

### Scatterplot

In [25]:
eu_idf = eu_df.interactive()

scatterplot_pipeline = (
    eu_idf[
        (eu_idf.year == year_slider) 
    ]
    .groupby(['Country', 'year', 'TFR'])['LifeExpectation'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [26]:
tfr_vs_life_scatterplot = scatterplot_pipeline.hvplot(x='TFR', y='LifeExpectation', by='Country', size=500, kind="scatter", title='Life Expectation vs. TFR', legend=False, alpha=0.7)#.opts(legend_position='right')
tfr_vs_life_scatterplot

BokehModel(combine_events=True, render_bundle={'docs_json': {'9f42378b-4e22-4b6e-84aa-39ac70b247ec': {'defs': …

### Choose either Population Curve or TFR
Using botton widget

In [27]:
ita_pop = df[df.sex == 'Both sexes'].groupby(['timeLabel', 'location'])['value'].sum().to_frame().reset_index().sort_values(by='timeLabel').reset_index(drop=True).rename(columns={'value':'Population'})

In [28]:
tfr = get_data(indicator = 19, # Total fertility rate
        location = 380 # Italy
        )[['location', 'timeLabel', 'value']].rename(columns={'value':'TFR'})

new_df = tfr.merge(ita_pop, right_on=['location', 'timeLabel'], left_on=['location', 'timeLabel'])

In [29]:
new_df = new_df.rename(columns={'timeLabel':'year', 'location':'Country'})

In [30]:
# rename for bottons
new_df = new_df.rename(columns={'TFR':'Total Fertility Rate (TFR) - Italy', 'Population':'Population curve - Italy'})

# botton
botton = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Total Fertility Rate (TFR) - Italy', 'Population curve - Italy'], 
    button_type='success'
)

# interactive dataframe based on botton choice
nidf = new_df.interactive()
botton_pipeline = (
    nidf[
        (nidf.year <= year_slider)
    ].groupby(['year', 'Country'])[botton].sum().to_frame().reset_index().sort_values(by='year').reset_index(drop=True)
)

# plot
curve_plot = botton_pipeline.hvplot(x='year', y=botton, by='Country', line_width=2, legend=False, color='#ef7a85') 
curve_plot 

BokehModel(combine_events=True, render_bundle={'docs_json': {'14cc6935-b216-4fe4-9ee1-71e8df1ce0be': {'defs': …

### Put all together in the panel dashboard

In [34]:
# Layout using Template
template = pn.template.FastListTemplate(
    title='Demographic trends dashboard', 
    sidebar=[pn.pane.Markdown("# Demographic changes"), 
             pn.pane.Markdown("#### From a demographic perspective, new challenges have arisen in recent years. The drop in the total fertility rate (TFR) and the ageing of the population lead to the need to implement new policies in the areas of welfare, pensions and childcare, as well as in the sphere of gender equality, proven to be essential to prevent the birth rate from falling.\nThe dashboard displays the demographic situation in Italy, enabling comparisons with other countries in Europe."), 
             pn.pane.JPG('imgs/wordcloud.jpg', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],

    main=[pn.Row(pn.Column(pn.bind(pyramid, year_slider)), # pn.bind to connect slider to pyramid plot
                 data_table2.panel(sizing_mode='stretch_width')),
            pn.Row(pn.Column(botton, curve_plot.panel(sizing_mode='stretch_width')), 
                    tfr_vs_life_scatterplot.panel(sizing_mode='stretch_width')) 
            ],

    accent_base_color="#245CC4",
    header_background="#245CC4",
)
template.show()
template.servable();